In [63]:
import json
from sentence_transformers import SentenceTransformer, util
from summarizer.sbert import SBertSummarizer
from transformers import BertTokenizer,AlbertTokenizer,AutoTokenizer, AlbertForQuestionAnswering, AutoModelForQuestionAnswering ,BertForQuestionAnswering

from transformers import BertTokenizer
import torch

from transformers import pipeline

In [27]:
data_size = 10
search_text_size = 128

In [28]:
articles = []
file = open('data/val.txt', encoding='utf-8', mode='r')
for i in range(data_size):
    line = file.readline()
    art = json.loads(line)
    abstract = " ".join([l.replace('<S>', '').replace('</S>', '') for l in art['abstract_text']])
    passage = " ".join(art['article_text'])
    articles.append([abstract, passage])
file.close()

In [41]:
model = SBertSummarizer('paraphrase-MiniLM-L6-v2')
encoder = SentenceTransformer('all-MiniLM-L6-v2')

In [45]:
# print(articles[0][1])
# print("================================")
abses = []
arts = []
for abs, art in articles:
    if len(abs.split()) > 256:
        abs = model(abs, min_length=search_text_size)
    if len(art.split()) > 256:
        art = model(art, min_length=search_text_size)
    abses.append(abs)
    arts.append(art)

arts_embeddings = encoder.encode(arts)
abs_embeddings = encoder.encode(abses)

C:\Users\Leo\AppData\Roaming\Python\Python39\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
C:\Users\Leo\AppData\Roaming\Python\Python39\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
C:\Users\Leo\AppData\Roaming\Python\Python39\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
C:\Users\Leo\AppData\Roaming\Python\Python39\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warning

In [47]:
abs_embeddings.shape
arts_embeddings.shape

(10, 384)

In [80]:
question = "who decides whether stroke status is correct?"
question_embedding = encoder.encode(question, convert_to_tensor=True)
hits = util.semantic_search(question_embedding, abs_embeddings, top_k=5)
reference_text = articles[hits[0][0]['corpus_id']][1]

In [81]:
hits

[[{'corpus_id': 1, 'score': 0.6062844395637512},
  {'corpus_id': 0, 'score': 0.28114214539527893},
  {'corpus_id': 2, 'score': 0.25351449847221375},
  {'corpus_id': 4, 'score': 0.2026546448469162},
  {'corpus_id': 7, 'score': 0.18032681941986084}]]

In [75]:
articles[1][0]

' backgroundthe questionnaire for verifying stroke - free status ( qvsfs ) has been validated in western populations as a method for verifying stroke - free status in participants of clinical , epidemiological and genetic studies .   this instrument has not been validated in low - income settings where populations have limited knowledge of stroke symptoms and literacy levels are low.objectiveto simultaneously validate the 8-item qvsfs in 3 languages spoken in west africa ( yoruba , hausa and akan ) for ascertainment of stroke - free status of control subjects in siren.methodsusing a cross - sectional study design , 100 participants each from the 3 linguistic groups will be consecutively recruited from neurology and general medicine clinics of 5 tertiary referral hospitals in nigeria and ghana .   ascertainment of stroke status will be determined by neurologists using structured neurological examination , review of case records and neuro - imaging ( gold standard ) .   the relative perf

In [65]:
question_answerer = pipeline("question-answering", model='distilbert-base-cased-distilled-squad')

In [86]:
# reference_text = articles[hits[0][0]['corpus_id']][1]
reference_text = arts[hits[0][0]['corpus_id']]
result = question_answerer(question=question, context=reference_text)

In [87]:
result

{'score': 0.01487144734710455,
 'start': 1467,
 'end': 1508,
 'answer': 'review of the medical records of patients'}